# 상장기업정보 수집 하기

In [1]:
import time
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
def info_extraction(stock_list):
    page_result = []
    for stock in stock_list:
        stock_name = stock.select_one(".vmiddle.legend")['alt']
        company_name = stock.select_one("#companysum").text
        code = stock.select_one("#companysum")['onclick'].split("'")[1]
        code = f"{code:0<6}"
        industry = stock.select_one("td:nth-child(2)").text 
        main_products = stock.select_one("td:nth-child(3)").text
        listing_date = stock.select_one("td:nth-child(4)").text
        fiscal_month = stock.select_one("td:nth-child(5)").text
        ceo_name = stock.select_one("td:nth-child(6)").text
        website = stock.select_one(".btn.ico.homepage")['href'] if stock.select_one(".btn.ico.homepage") != None else "없음"
        region = stock.select_one("td:nth-child(8)").text
        values = (stock_name, company_name, code, industry, main_products, listing_date, fiscal_month, ceo_name, website, region)
        page_result.append(values)
    return page_result

In [3]:
url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method="searchCorpList",pageIndex=1,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
r = requests.post(url, data = payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, 'lxml')
soup

https://kind.krx.co.kr/corpgeneral/corpList.do
200


<html><body><section class="scrarea type-00">
<table class="list type-00 tmt30" summary="회사명, 업종, 주요제품, 상장일, 결산월, 대표자명, 홈페이지, 지역">
<caption>목록</caption>
<colgroup>
<col width="*"/>
<col width="15%"/>
<col width="20%"/>
<col width="11%"/>
<col width="8%"/>
<col width="10%"/>
<col width="8%"/>
<col width="12%"/>
</colgroup>
<thead>
<tr class="first" id="title-contents">
</tr>
</thead>
<tbody>
<tr class="first">
<td class="first" title="명인제약(주)"><img alt="유가증권" class="vmiddle legend" src="/images/common/icn_t_yu.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('31745'); return false;" title="명인제약"> 명인제약</a> </td>
<td class="textOverflow" title="의약품 제조업">의약품 제조업</td>
<td class="textOverflow" title="신경정신계열, 구강계열 완제의약품">신경정신계열, 구강계열 완제의약품</td>
<td class="txc">2025-10-01</td>
<td class="txc">12월</td>
<td class="txc" title="이행명">이행명</td>
<td class="txc">
<a class="btn ico homepage" href="http://www.myunginph.co.kr" target="_blank" title="http://www.myunginph.co.kr"><sp

In [38]:
int(soup.select_one(".info.type-00 > em").text.replace(",","")) // 100 +1

28

In [8]:
stock_list = soup.select("tbody > tr")
stock_list[0]

<tr class="first">
<td class="first" title="명인제약(주)"><img alt="유가증권" class="vmiddle legend" src="/images/common/icn_t_yu.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('31745'); return false;" title="명인제약"> 명인제약</a> </td>
<td class="textOverflow" title="의약품 제조업">의약품 제조업</td>
<td class="textOverflow" title="신경정신계열, 구강계열 완제의약품">신경정신계열, 구강계열 완제의약품</td>
<td class="txc">2025-10-01</td>
<td class="txc">12월</td>
<td class="txc" title="이행명">이행명</td>
<td class="txc">
<a class="btn ico homepage" href="http://www.myunginph.co.kr" target="_blank" title="http://www.myunginph.co.kr"><span>홈페이지 보기</span></a>
</td>
<td class="txc">경기도</td>
</tr>

In [9]:
# 증권종류
stock_name = soup.select_one(".vmiddle.legend")['alt']

In [10]:
# 회사명
company_name = soup.select_one("#companysum").text

In [11]:
# 종목코드
code = soup.select_one("#companysum")['onclick'].split("'")[1]
code = f"{code:0<6}"
code

'317450'

In [12]:
# 업종
industry = soup.select_one("td:nth-child(2)").text
industry

'의약품 제조업'

In [13]:
# 주요제품
main_products = soup.select_one("td:nth-child(3)").text
main_products

'신경정신계열, 구강계열 완제의약품'

In [14]:
# 상장일
listing_date = soup.select_one("td:nth-child(4)").text
listing_date

'2025-10-01'

In [15]:
# 결산월
fiscal_month = soup.select_one("td:nth-child(5)").text

In [16]:
# 대표자명
ceo_name = soup.select_one("td:nth-child(6)").text

In [17]:
# 홈페이지
website = soup.select_one(".btn.ico.homepage")['href']

In [18]:
# 지역
region = soup.select_one("td:nth-child(8)").text

In [19]:
result = {}
keys = ['주식종목', '회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

for idx, stock in enumerate(soup):
    stock_name = stock.select_one(".vmiddle.legend")['alt']
    company_name = stock.select_one("#companysum").text
    code = stock.select_one("#companysum")['onclick'].split("'")[1]
    code = f"{code:0<6}"
    industry = stock.select_one("tbody > tr.first > td:nth-child(2)").text
    main_products = stock.select_one("tbody > tr.first > td:nth-child(3)").text
    listing_date = stock.select_one("tbody > tr.first > td:nth-child(4)").text
    fiscal_month = stock.select_one("tbody > tr.first > td:nth-child(5)").text
    ceo_name = stock.select_one("tbody > tr.first > td:nth-child(6)").text
    website = stock.select_one(".btn.ico.homepage")['href'] if stock.select_one(".btn.ico.homepage")['href'] != None else "없음"
    region = stock.select_one("tbody > tr.first > td:nth-child(8)").text
    
    values = (stock_name, company_name, code, industry, main_products, listing_date, fiscal_month, ceo_name, website, region)
    
    for key, value in zip(keys, values):
        result.setdefault(key, []).append(value)
        
result
df = pd.DataFrame(result)
df

주식종목    회사명    종목코드       업종                주요제품         상장일  결산월 대표자명  \
0  유가증권   명인제약  317450  의약품 제조업  신경정신계열, 구강계열 완제의약품  2025-10-01  12월  이행명   

                         홈페이지   지역  
0  http://www.myunginph.co.kr  경기도

# 전체 페이지

In [36]:
result = {}
for page in range(1,29):
    
    print(f"{page}/28", end="\r")
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList",pageIndex=page,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data = payload)
    soup = bs(r.content, 'lxml')
    
    stock_list = soup.select("table.list.type-00.tmt30 > tbody > tr")

    keys = ['주식종목', '회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

    for idx, stock in enumerate(stock_list):
        stock_name = stock.select_one(".vmiddle.legend")['alt']
        company_name = stock.select_one("#companysum").text
        code = stock.select_one("#companysum")['onclick'].split("'")[1]
        code = f"{code:0<6}"
        industry = stock.select_one("td:nth-child(2)").text 
        main_products = stock.select_one("td:nth-child(3)").text
        listing_date = stock.select_one("td:nth-child(4)").text
        fiscal_month = stock.select_one("td:nth-child(5)").text
        ceo_name = stock.select_one("td:nth-child(6)").text
        website = stock.select_one(".btn.ico.homepage")['href'] if stock.select_one(".btn.ico.homepage") != None else "없음"
        region = stock.select_one("td:nth-child(8)").text

        values = (stock_name, company_name, code, industry, main_products, listing_date, fiscal_month, ceo_name, website, region)

        for key, value in zip(keys, values):
            result.setdefault(key, []).append(value)
        
result
df = pd.DataFrame(result)
df

주식종목        회사명    종목코드                   업종  \
0     유가증권       명인제약  317450              의약품 제조업   
1      코스닥   KB제33호스팩  0072Z0           금융 지원 서비스업   
2      코스닥    삼성스팩11호  0071M0           금융 지원 서비스업   
3      코스닥     에스투더블유  488280       소프트웨어 개발 및 공급업   
4      코스닥         오아  342870    가전제품 및 정보통신장비 소매업   
...    ...        ...     ...                  ...   
2758  유가증권       유한양행  000100              의약품 제조업   
2759  유가증권     CJ대한통운  000120            도로 화물 운송업   
2760  유가증권         경방  000050               종합 소매업   
2761  유가증권      유수홀딩스  000700  회사 본부 및 경영 컨설팅 서비스업   
2762  유가증권   한진중공업홀딩스  003480    연료용 가스 제조 및 배관공급업   

                                                   주요제품         상장일  결산월  \
0                                    신경정신계열, 구강계열 완제의약품  2025-10-01  12월   
1                                                기업인수합병  2025-09-30  12월   
2                                                기업인수합병  2025-09-30  12월   
3     공공/정부기관용 사이버안보 빅데이터 플랫폼, 기업 및 기관용 사이버위협 인텔리전스 ...  2025-09-19  12월   
4                               계절/생활/주방/건강가전 및 헬스/건강식품  2025-09-09  12월   
...                                                 ...         ...  ...   
2758  의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...  1962-11-01  12월   
2759  Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...  1956-07-02  12월   
2760          섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입  1956-03-03  12월   
2761                                               지주사업  1956-03-03  12월   
2762                                               지주회사  1956-03-03  12월   

       대표자명                          홈페이지     지역  
0       이행명    http://www.myunginph.co.kr    경기도  
1       박성원                            없음  서울특별시  
2       문소연                            없음  서울특별시  
3       서상덕               http://s2w.inc/    경기도  
4       김상무       http://www.oa-world.com  서울특별시  
...     ...                           ...    ...  
2758  대표이..        http://www.yuhan.co.kr  서울특별시  
2759  신영수..    http://www.cjlogistics.com  서울특별시  
2760  김준,..    http://www.kyungbang.co.kr  서울특별시  
2761    송영규  http://www.eusu-holdings.com  서울특별시  
2762    조원국  http://www.hhic-holdings.com    경기도  

[2763 rows x 10 columns]

In [49]:
result = {}    
cols = ['주식종목', '회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

page = 1
total_page = 1

while page <= total_page:
    print(f"{page}/{total_page}", end="\r")
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList",pageIndex=page,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data = payload)
    # print(r.status_code)
    soup = bs(r.content, 'lxml')
    if total_page == 1:
        total_page = int(soup.select_one(".info.type-00 > em").text.replace(",","")) // 100 +1

    page_result = info_extraction(soup.select("tbody > tr"))
    for items in page_result:
        for key, value in zip(cols, items):
            result.setdefault(key, []).append(value)
            
    page += 1
    time.sleep(1)
        
df = pd.DataFrame(result)
df

주식종목        회사명    종목코드                   업종  \
0     유가증권       명인제약  317450              의약품 제조업   
1      코스닥   KB제33호스팩  0072Z0           금융 지원 서비스업   
2      코스닥    삼성스팩11호  0071M0           금융 지원 서비스업   
3      코스닥     에스투더블유  488280       소프트웨어 개발 및 공급업   
4      코스닥         오아  342870    가전제품 및 정보통신장비 소매업   
...    ...        ...     ...                  ...   
2758  유가증권       유한양행  000100              의약품 제조업   
2759  유가증권     CJ대한통운  000120            도로 화물 운송업   
2760  유가증권         경방  000050               종합 소매업   
2761  유가증권      유수홀딩스  000700  회사 본부 및 경영 컨설팅 서비스업   
2762  유가증권   한진중공업홀딩스  003480    연료용 가스 제조 및 배관공급업   

                                                   주요제품         상장일  결산월  \
0                                    신경정신계열, 구강계열 완제의약품  2025-10-01  12월   
1                                                기업인수합병  2025-09-30  12월   
2                                                기업인수합병  2025-09-30  12월   
3     공공/정부기관용 사이버안보 빅데이터 플랫폼, 기업 및 기관용 사이버위협 인텔리전스 ...  2025-09-19  12월   
4                               계절/생활/주방/건강가전 및 헬스/건강식품  2025-09-09  12월   
...                                                 ...         ...  ...   
2758  의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...  1962-11-01  12월   
2759  Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...  1956-07-02  12월   
2760          섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입  1956-03-03  12월   
2761                                               지주사업  1956-03-03  12월   
2762                                               지주회사  1956-03-03  12월   

       대표자명                          홈페이지     지역  
0       이행명    http://www.myunginph.co.kr    경기도  
1       박성원                            없음  서울특별시  
2       문소연                            없음  서울특별시  
3       서상덕               http://s2w.inc/    경기도  
4       김상무       http://www.oa-world.com  서울특별시  
...     ...                           ...    ...  
2758  대표이..        http://www.yuhan.co.kr  서울특별시  
2759  신영수..    http://www.cjlogistics.com  서울특별시  
2760  김준,..    http://www.kyungbang.co.kr  서울특별시  
2761    송영규  http://www.eusu-holdings.com  서울특별시  
2762    조원국  http://www.hhic-holdings.com    경기도  

[2763 rows x 10 columns]

In [50]:
df.to_csv("./data/stock_list.csv", index=False, encoding="utf-8-sig")

In [25]:
from io import StringIO

In [47]:
df_list = []
for page in range(1,28):
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList",pageIndex=page,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data = payload)
    soup = bs(r.content, 'lxml')
    temp = pd.read_html(StringIO(r.text))
    df_list.append(temp[0])

df2=pd.concat(df_list)


In [48]:
df2.columns = ['회사명', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']
df2

회사명                 업종  \
0       명인제약            의약품 제조업   
1   KB제33호스팩         금융 지원 서비스업   
2    삼성스팩11호         금융 지원 서비스업   
3     에스투더블유     소프트웨어 개발 및 공급업   
4         오아  가전제품 및 정보통신장비 소매업   
..       ...                ...   
95     일동홀딩스            의약품 제조업   
96        대동      특수 목적용 기계 제조업   
97    오리온홀딩스             기타 금융업   
98     깨끗한나라    펄프, 종이 및 판지 제조업   
99      태원물산      자동차 신품 부품 제조업   

                                                 주요제품         상장일  결산월   대표자명  \
0                                  신경정신계열, 구강계열 완제의약품  2025-10-01  12월    이행명   
1                                              기업인수합병  2025-09-30  12월    박성원   
2                                              기업인수합병  2025-09-30  12월    문소연   
3   공공/정부기관용 사이버안보 빅데이터 플랫폼, 기업 및 기관용 사이버위협 인텔리전스 ...  2025-09-19  12월    서상덕   
4                             계절/생활/주방/건강가전 및 헬스/건강식품  2025-09-09  12월    김상무   
..                                                ...         ...  ...    ...   
95                                          의약품 제조,도매  1975-06-28  12월    박대창   
96          경운기,트랙터,이앙기,바인더,수확기(콤바인),농업용엔진 및 부품 제조,판매  1975-06-27  12월  김준식..   
97                                      자회사 관리 지주사업 등  1975-06-27  12월    허인철   
98                                 펄프,판지원지,화장지 제조,수출입  1975-06-25  12월  최현수..   
99              자동차부품,워터펌프,인산정제석고,건축자재(불연내장재), 제조,수출입  1975-06-24  12월    남기영   

       홈페이지     지역  
0   홈페이지 보기    경기도  
1       NaN  서울특별시  
2       NaN  서울특별시  
3   홈페이지 보기    경기도  
4   홈페이지 보기  서울특별시  
..      ...    ...  
95  홈페이지 보기  서울특별시  
96  홈페이지 보기  대구광역시  
97  홈페이지 보기  서울특별시  
98  홈페이지 보기  서울특별시  
99  홈페이지 보기  서울특별시  

[2700 rows x 8 columns]